In [96]:
# 导入模块
import pandas as pd
import numpy as np
from sklearn import ensemble, preprocessing, metrics
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
import time

In [97]:
# 使用数据集
train0 = pd.read_csv('train.csv')
weather = pd.read_csv('weather.csv')
test0 = pd.read_csv('test.csv')
# DataFrame 是一个表格型的数据结构，它含有一组有序的列，每列可以是不同的值类型（数值、字符串、布尔型值）。
# DataFrame 既有行索引也有列索引，它可以被看做由 Series 组成的字典（共同用一个索引）。

In [98]:
# 按照时间顺序将train0数据集划分为train，test数据集

# 思考：如何使用字符串匹配来划分train、test数据集？

train = train0.copy()
test = test0.copy()

In [99]:
# 处理weather数据集
def weather_process(dataframe):
    dataframe = dataframe.drop('CodeSum', axis=1)
    dataframe = dataframe.replace('-', -1)
    dataframe = dataframe.replace('M', -1)
    dataframe = dataframe.replace('  T', -1)
    return dataframe

In [100]:
# 处理train，test数据集
def train_process(shit):

    shit['Month'] = shit.Date.apply(create_month)
    shit['Day'] = shit.Date.apply(create_day)
    shit['Lat_appro'] = shit.Latitude.apply(appro)
    shit['Long_appro'] = shit.Longitude.apply(appro)
    shit = shit.drop(['Address', 'Block', 'Street'], axis=1)
    return shit

In [101]:
# 分割日期数据

def create_month(x):
    return x.split('-')[1]

def create_day(x):
    return x.split('-')[2]

In [102]:
# 取经纬度的近似值

def appro(x):
    return round(x, 2)

In [103]:
# 根据日期将天气数据与训练集测试集合并

def join_process(data, weather):
    data = data.merge(weather, on='Date')
    data = data.drop(['Date'], axis=1)
    return data

In [104]:
# 预处理数据，将标签编码，得到物种、位置以及陷阱对应的编码值

def encode_data(dataframe):
    lbl = preprocessing.LabelEncoder()
    sp = list(train0['Species'].values)+list(test0['Species'].values)
    lbl.fit(sp) #
    dataframe['Species'] = lbl.transform(dataframe['Species'].values)
    
    tr = list(train0['Trap'].values)+list(test0['Trap'].values)
    lbl.fit(tr)#
    dataframe['Trap'] = lbl.transform(dataframe['Trap'].values)
    
    ad = list(train0['AddressNumberAndStreet'].values)+list(test0['AddressNumberAndStreet'].values)
    lbl.fit(ad)#
    dataframe['AddressNumberAndStreet'] = lbl.transform(dataframe['AddressNumberAndStreet'].values)
    return dataframe

In [105]:
# 剔除坏数据，切片得到有效数据

def slice_data(dataframe):
    label = dataframe.WnvPresent.values
    train = dataframe.drop(['WnvPresent'], axis=1)
    return [train,label]

In [106]:

train = train_process(train)
train = join_process(train, weather_process(weather))
train = encode_data(train )
train = slice_data(train)

In [107]:
test = encode_data(join_process(train_process(test), weather_process(weather)))

In [108]:
# 训练随机森林模型

e = time.time()
clf = ensemble.RandomForestClassifier(n_jobs=-1, n_estimators=1000, min_samples_split=50)
clf.fit(train[0], train[1])
s = time.time()

In [109]:
# 计算训练用时

print(clf.classes_)
print('training time: '+str(s-e) + 's')

[0 1]
training time: 2.8116142749786377s


In [121]:
predictions = pd.DataFrame(clf.predict(test))
predictions.index.name = 'ID'
predictions.columns = ['WnvPresent']

C:\ProgramData\Anaconda3\envs\BD1Oct26\lib\site-packages\sklearn\base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Id
Feature names seen at fit time, yet now missing:
- NumMosquitos

  warnings.warn(message, FutureWarning)


In [122]:
predictions.to_csv('MysampleSubmission.csv')